In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import os
from os.path import exists
import time
import sys
sys.path.insert(1, '../../rtsvg')
from rtsvg import *
rt = RACETrack()
#os.listdir('../../../data/2014_vast/MC2/Geospatial')
#_shaplely_lu_, _shaplely_fill_lu_ = rt.createBackgroundLookupsFromShapeFile('../../../data/2014_vast/MC2/Geospatial/Kronos_Island.shp')
#_shaplely_lu_[0]

In [ ]:
_conversion_str_ = '''
df = pl.read_csv('../../../data/2014_vast/MC2/gps.csv')
_datetimes_ = []
for i in range(len(df)):
    _date_ = df['Timestamp'][i]
    try:
        _datetime_ = pd.to_datetime(_date_)
        _datetimes_.append(_datetime_)
    except:
        print(i)
df_fixed = pd.DataFrame({'timestamp':_datetimes_, 'id':df['id'], 'lat':df['lat'], 'long':df['long']})
df_fixed = rt.columnsAreTimestamps(df_fixed, 'timestamp')
df_fixed_pl = pl.DataFrame(df_fixed)
df_fixed_pl.write_parquet('../../../data/2014_vast/MC2/gps.csv.parquet')
'''
df = pl.read_parquet('../../../data/2014_vast/MC2/gps.csv.parquet')
df.sample(3)

In [ ]:
# One of the paths is messed up / it's green in the following visualization
#rt.xy(df, x_field='long', y_field='lat', color_by='id', dot_size='small', w=800, h=600)

In [ ]:
# path that is messed up is the following:
# id == 28 using the following interaction panel
spec = { 
    (0,0,20,20): ('xy', {'x_field':'long','y_field':'lat','dot_size':'small'}),
    (20,0,5,20): ('histogram', {'bin_by':'id'})
}
#rt.interactivePanel(df_not_id28, spec, 1000, 600, {'color_by':'id'})

In [ ]:
df_id28     = df.filter(pl.col('id') == 28)
df_id27     = df.filter(pl.col('id') == 27)
df_not_id28 = df.filter(pl.col('id') != 28)
#rt.tile([rt.xy(df_id28,     x_field='long', y_field='lat', color_by='id', dot_size='small', w=600, h=600),
#         rt.xy(df_not_id28, x_field='long', y_field='lat', color_by='id', dot_size='small', w=800, h=600)])

In [ ]:
df_id27_sorted = df_id27.sort('timestamp')
_df_ = df_id27_sorted
_df_.sample(3)

In [ ]:
#
# The VAST 2014 geospatial data lacks the precision to calculate
# miles per hour on a row by row basis... because it's only at the
# seconds level... and not the milliseconds level..
#

# template to calculate miles per hour... would still need smoothing
# because the second by second capture introduces some error....
#
import math
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees).
    Source: https://gis.stackexchange.com/a/56589/15183
    """
    if lon1 is None or lat1 is None or lon2 is None or lat2 is None: return 0.0
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    km = 6367 * c
    return km * 0.621371 # km to miles

_df_ = _df_.with_columns(pl.col('timestamp').shift().alias('prev_timestamp'),
                         pl.col('lat').shift().alias('prev_lat'),
                         pl.col('long').shift().alias('prev_long'))
_df_ = _df_.with_columns(pl.struct(['lat','long','prev_lat','prev_long'])
                           .map_elements(lambda x: haversine(x['long'],x['lat'],x['prev_long'],x['prev_lat']), return_dtype=pl.Float64).alias('miles'))
_df_ = _df_.with_columns(((pl.col('timestamp') - pl.col('prev_timestamp'))/3600000.0).alias('t_delta'))
def mph(miles, hours): # not enough temporal resolution for this to work
    if miles is None or hours is None or hours == 0.0: return 0.0
    return miles
    #return miles/hours
_df_ = _df_.with_columns(pl.struct(['miles','t_delta']).map_elements(lambda x: mph(x['miles'],x['t_delta']), return_dtype=pl.Float64).alias('mph')) 
#rt.xy(_df_, x_field='long', y_field='lat', count_by='mph', dot_size='small', color_magnitude='mean', w=600, h=600)

In [ ]:
_df_ = df_not_id28.with_columns((pl.col('lat')  * 20000).cast(pl.Int64).alias('lat_grid'),
                                (pl.col('long') * 20000).cast(pl.Int64).alias('long_grid'))
#rt.xy(_df_, x_field='long_grid', y_field='lat_grid', color_by='id', dot_size='tiny', w=1200, h=800)

In [ ]:
lu  = {'fm':[], 'to':[]}
pos = {}
def toNodeName(_tuple_): return 'node_' + str(_tuple_[0]) + '_' + str(_tuple_[1])
for k, k_df in _df_.sort(['id','timestamp']).group_by('id', maintain_order=True):
    for i in range(1, len(k_df)-1):
        _fm_tuple_ = (k_df['long_grid'][i],   k_df['lat_grid'][i])
        _fm_       = toNodeName(_fm_tuple_)
        _to_tuple_ = (k_df['long_grid'][i+1], k_df['lat_grid'][i+1])
        _to_       = toNodeName(_to_tuple_)
        lu['fm'].append(_fm_)
        lu['to'].append(_to_)
        if _fm_ not in pos: pos[_fm_] = _fm_tuple_
        if _to_ not in pos: pos[_to_] = _to_tuple_
#rt.link(pl.DataFrame(lu), [('fm','to')], pos, node_size=1.0, w=1200, h=800)

In [ ]:
lat_rez, long_rez = 1500, 1500
_df_ = df_not_id28.with_columns((pl.col('lat')  * lat_rez) .cast(pl.Int64).alias('lat_grid'),
                                (pl.col('long') * long_rez).cast(pl.Int64).alias('long_grid'))
_df_ = _df_.sort(['id','timestamp'])
_lu_      = {'id':[], 'long_grid':[], 'lat_grid':[], 'timestamp':[]}
for k, k_df in _df_.group_by(['id'], maintain_order=True):
    for l, l_df in k_df.group_by(['lat_grid','long_grid', 'timestamp'], maintain_order=True):
        _lu_['id'].append(l_df['id'][0])
        _lu_['long_grid'].append(l_df['long_grid'][0])
        _lu_['lat_grid'].append(l_df['lat_grid'][0])
        _lu_['timestamp'].append(l_df['timestamp'][0])

df_grid_trans = pl.DataFrame(_lu_)
df_grid_trans = df_grid_trans.with_columns(pl.col('long_grid').shift().alias('long_grid_last'),
                                           pl.col('lat_grid'). shift().alias('lat_grid_last'),
                                           pl.col('timestamp').shift().alias('timestamp_last'),
                                           pl.col('id').shift().alias('id_last'))
df_grid_trans = df_grid_trans.with_columns((    pl.col('timestamp') - pl.col('timestamp_last')). alias('t_delta'),
                                           (abs(pl.col('long_grid') - pl.col('long_grid_last'))).alias('long_inc'),
                                           (abs(pl.col('lat_grid')  - pl.col('lat_grid_last'))). alias('lat_inc'),
                                           (abs(pl.col('id')        - pl.col('id_last'))      ). alias('id_inc'))

df_grid_trans = df_grid_trans.filter((pl.col('long_inc') < 8) & (pl.col('lat_inc') < 8) & (pl.col('id_inc') == 0))
df_grid_trans = df_grid_trans.sort('long_inc')
for k, k_df in df_grid_trans.group_by('long_inc', maintain_order=True): print('long_inc', k[0], len(k_df))
print()
df_grid_trans = df_grid_trans.sort('lat_inc')
for k, k_df in df_grid_trans.group_by('lat_inc'):  print('lat_inc',  k[0], len(k_df))

In [ ]:
pos = {}
lu  = {'fm':[], 'to':[]}
for k, k_df in df_grid_trans.group_by(['long_grid','lat_grid','long_grid_last','lat_grid_last']):
    _fm_ = str(k[2]) + '_' + str(k[3])
    _to_ = str(k[0]) + '_' + str(k[1])
    lu['fm'].append(_fm_)
    lu['to'].append(_to_)
    if _fm_ not in pos: pos[_fm_] = (k[2], k[3])
    if _to_ not in pos: pos[_to_] = (k[0], k[1])
rt.link(pl.DataFrame(lu), [('fm','to')], pos, node_size=1.0, w=1200, h=800)